# 1. 폴더 정리

* 제공해드린 animals10.zip의 압축을 풀어 그 내용을 살펴보시고, 폴더 구조를 학습에 알맞도록 재구성해주세요
* 특히 각 클래스마다 약 30% 정도의 이미지를 test 폴더에 할당해주세요
* 중간중간 헷갈리다면 이것저것 확인하는 코드를 거쳐보세요 (ex. flat_test[:5]로 앞의 다섯값 확인)
* 궁금한 점이 있을 경우, 슬랙 질문 채널 활용을 적극 권장합니다.

In [ ]:
# colab 사용할 경우
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 현재 디렉터리 위치 확인
!pwd

/content


In [ ]:
# python에서 파일 관리해주는 라이브러리
import os
import glob
import shutil

In [ ]:
### Guideline for convenience ###
# 압축 풀기
!unzip -qq "/content/drive/MyDrive/animals10.zip"

replace Animals-10/butterfly/butterfly (1).jpeg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [ ]:
!ls

Animals-10  drive  sample_data	test  train


In [ ]:
# 각 클래스별 파일 개수 확인 -> 변수로 해당 개수 저장
# Tip) 클래스가 10개이므로 각 클래스마다 똑같은 내용의 코드를 계속 써야할까요? 클래스 이름을 리스트로 저장해 for문을 돌리면서 코드를 재사용하는건 어떨까요?

animals = ['butterfly', 'cat', 'chicken', 'cow', 'dog', 'elephant', 'horse', 'sheep', 'spider', 'squirrel']
for i in range(10):
    globals()[animals[i]] = sorted(glob.glob(f'./Animals-10/{animals[i]}/*'))


In [ ]:
for i in range(10):
    print(len(globals()[animals[i]]))

2112
1668
3098
1866
4863
1446
2623
1820
4821
1862


In [ ]:
cat[:5]

['./Animals-10/cat/cat (1).jpeg',
 './Animals-10/cat/cat (1).jpg',
 './Animals-10/cat/cat (1).png',
 './Animals-10/cat/cat (10).jpeg',
 './Animals-10/cat/cat (10).jpg']

In [ ]:
# test에 넣을 이미지 개수 계산
for i in range(10):
    globals()[f'{animals[i]}_test_count'] = round(len(globals()[animals[i]])*0.3)


In [ ]:
import random

def split(img_list, test_count, train_path, test_path):
  # img_list : 이미지 경로 리스트
  # test_count : test에 할당할 이미지 개수
  # train_path : train 데이터 넣을 경로
  # test_path : test 데이터 넣을 경로
  
  # 랜덤으로 test_count만큼 이미지 img_list에서 고르기
  # test 담을 이미지 리스트 저장
  test_files=[]
  for i in random.sample(img_list, test_count):
    test_files.append(i)

  # 위에서 고르지 않은 이미지들을 train 담을 이미지 리스트로 저장
  train_files = [x for x in img_list if x not in test_files]

  # 고른 이미지를 train_path, test_path폴더에 폭사
  for k in train_files:
    shutil.copy(k, train_path)
  
  for c in test_files:
    shutil.copy(c, test_path)

  print('train 폴더 이미지 개수 : {}\ntest 폴더 이미지 개수 : {}'.format(len(glob.glob(train_path+'/*')),len(glob.glob(test_path+'/*'))))

In [ ]:
# train, test 폴더 경로 선언 & 만들기

for i in range(10):
    globals()[f'{animals[i]}_train_path'] = f'./train/{animals[i]}/'
    globals()[f'{animals[i]}_test_path'] = f'./test/{animals[i]}/'
    os.makedirs(globals()[f'{animals[i]}_train_path'], exist_ok=True)
    os.makedirs(globals()[f'{animals[i]}_test_path'], exist_ok=True)

In [ ]:
# train, test 폴더로 이미지 split해서 알맞게 집어넣기

for i in range(10):
    split(globals()[animals[i]], globals()[f'{animals[i]}_test_count'], globals()[f'{animals[i]}_train_path'],  globals()[f'{animals[i]}_test_path'])

train 폴더 이미지 개수 : 1922
test 폴더 이미지 개수 : 1078
train 폴더 이미지 개수 : 1516
test 폴더 이미지 개수 : 848
train 폴더 이미지 개수 : 2808
test 폴더 이미지 개수 : 1568
train 폴더 이미지 개수 : 1701
test 폴더 이미지 개수 : 955
train 폴더 이미지 개수 : 4406
test 폴더 이미지 개수 : 2461
train 폴더 이미지 개수 : 1318
test 폴더 이미지 개수 : 740
train 폴더 이미지 개수 : 2400
test 폴더 이미지 개수 : 1351
train 폴더 이미지 개수 : 1645
test 폴더 이미지 개수 : 917
train 폴더 이미지 개수 : 4389
test 폴더 이미지 개수 : 2460
train 폴더 이미지 개수 : 1692
test 폴더 이미지 개수 : 948


# 2. CNN 모델
수업 코드에 '모델 성능에 대한 고민!!' 부분을 참고해서 **test 이미지 기준 최소 80% 이상**의 모델이 나올 수 있도록 학습을 진행해주세요.
* 중간중간 헷갈리다면 이것저것 확인하는 코드를 거쳐보세요
* 90% 이상의 모델이 이상적이긴 합니다
* 궁금한 점이 있을 경우, 슬랙 질문 채널 활용을 적극 권장합니다.

전반적으로 수업 코드와 비슷하게 짜면 되는데, 생각해봐야 할 것은
* 모든 이미지를 내가 정의하는 하나의 모델에 넣어야함
* 그말은 input 데이터의 차원이 항상 동일해야 된다는 말
* 그런데 과연 내가 가지고 있는 이미지들의 사이즈가 모두 같을까? - 대략 얼마정도 사이즈 가지고 있는지 코드로 확인해보면 더 좋음 (shape 확인)
* 이미지 사이즈를 통일시키기 위해서는 어떻게 해야할까? - transforms의 Resize, RandomCrop 써볼까?
* 동물 이미지에 적합한 Augmentation은 무엇이 있을까?

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [ ]:
# train, test 이미지를 위한 transforms function 정의
# 위에서 말한 포인트들 생각해보기 - 차원에 유의하자!

trans_tr = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.ColorJitter(contrast=0.5),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(20),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

In [ ]:
trans_te = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

In [ ]:
# ImageFolder와 앞서 정의한 transforms function을 활용해 Dataset 객체 (train, test에 대해) 선언

train_data = torchvision.datasets.ImageFolder(
    root = './train', 
    transform = trans_tr 
)
test_data = torchvision.datasets.ImageFolder(
    root = './test',
    transform = trans_te
)

In [ ]:
train_data[0][0].shape

torch.Size([3, 224, 224])

In [ ]:

# train, test에 대해 DataLoader 정의

train_loader = torch.utils.data.DataLoader(
    train_data,
    batch_size = 32,      # 일반적으로 batch size = 16, 32
    shuffle=True,         # train dataloader는 epoch마다 데이터 다시 섞어서 batch 만들고 학습! - 학습 효율 up
    num_workers=2
)

test_loader = torch.utils.data.DataLoader(
    train_data,
    batch_size = 32,      # 일반적으로 batch size = 16, 32
    shuffle=False,        # test dataloader는 데이터 매번 섞을 이유가 없음 - 어차피 확인 용도
    num_workers=2
)

In [ ]:
import torch.nn as nn

In [ ]:
### Model 클래스 정의 - 차원에 유의하자!
# 10개의 클래스를 분류해야 하는 꽤나 복잡한 task - 모델 구조를 어느 정도로 복잡하게 짜볼까?
# 가장 마지막 Linear의 out_features는 얼마로 해야할까?

class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        
        ##### Layer 정의 #####
        self.layer = nn.Sequential(
            # 맨처음 RGB 채널 3개이므로 가장 처음 in_channels = 3
            # img의 가장 첫 차원이 batch_size 값은 계속해서 유지
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=5),             # [batch_size,3,224,224] -> [batch_size,16,220,220] -> same with? - using 16 filters
            nn.ReLU(),
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5),            # [batch_size,16,220,220] -> [batch_size,32,216,216]
            nn.ReLU(),
            nn.Dropout(0.2), # 0.2 확률로 Dropout
            nn.MaxPool2d(kernel_size=2, stride=2),                                # [batch_size,32,216,216] -> [batch_size,32,108,108]
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=5, padding=2), # [batch_size,32,108,108] -> [batch_size,32,108,108] -> padding*2 주의!
            nn.ReLU(),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=5),            # [batch_size,32,108,108] -> [batch_size,32,104,104]
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.MaxPool2d(kernel_size=2, stride=2)                                 # [batch_size,32,104,104] -> [batch_size,32,52,52]
        )
        self.fc_layer = nn.Sequential(
            nn.Linear(32*52*52,100),                                                # [batch_size,32*52*52] -> [batch_size,100]
            nn.ReLU(),
            nn.Linear(100,10)
        )

    def forward(self, x):
        # forward pass 과정
        out = self.layer(x)
        out = out.view(out.size(0),-1)  # FC layer에 들어가기전, flatten! - view 함수 활용
                                        # 가장 첫 차원인 batch_size는 유지하고 나머지 차원들을 하나로 합치기 - [batch_size,16,5,5] -> [batch_size,16*5*5]
        out = self.fc_layer(out)
        return out

In [ ]:
# torchsummary로 모델 차원 어떻게 나오는지 돌려보기

from torchsummary import summary
model = MyModel().cuda() # 모델 객체 선언
summary(model, (3,224,224), batch_size=32)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [32, 16, 220, 220]           1,216
              ReLU-2         [32, 16, 220, 220]               0
            Conv2d-3         [32, 32, 216, 216]          12,832
              ReLU-4         [32, 32, 216, 216]               0
           Dropout-5         [32, 32, 216, 216]               0
         MaxPool2d-6         [32, 32, 108, 108]               0
            Conv2d-7         [32, 32, 108, 108]          25,632
              ReLU-8         [32, 32, 108, 108]               0
            Conv2d-9         [32, 32, 104, 104]          25,632
             ReLU-10         [32, 32, 104, 104]               0
          Dropout-11         [32, 32, 104, 104]               0
        MaxPool2d-12           [32, 32, 52, 52]               0
           Linear-13                  [32, 100]       8,652,900
             ReLU-14                  [

In [ ]:
# Device 선언 (GPU 권장) => 코랩 사용중이어서

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [ ]:
# loss func, optimizer 정의

criterion = nn.CrossEntropyLoss().to(device) # criterion (loss func)도 device 위에서
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001) # 보통 Adam의 learning rate로 0.001 사용
                                                                                  # weight_decay = L2 Regularization의 lambda값 (가중치 제한 정도)

In [ ]:
with torch.no_grad():
    for i, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.cuda(), targets.cuda()
        print(inputs.shape)  # batch_size개의 이미지
        print(targets.shape) # batch_size개의 클래스

        outputs = model(inputs)
        print(outputs.shape) # outputs는 32, 10 차원 (32개 data의 확률 10개씩)
        print(targets.shape) # targets는 32차원 (32개 data의 실제 클래스)

        loss = criterion(outputs, targets)  # -> cross entropy 식에 넣을때 이런 형태로 넣으면됨 (outputs, targets의 차원을 맞출 필요가 없음)
        print(loss.item())

        print(outputs.max(1)[1]) # 예측한 클래스
        print(targets) # 실제 클래스
        correct = (outputs.max(1)[1] == targets).sum().item() # 맞게 예측한 클래스 개수
        print(correct)
        break

torch.Size([32, 3, 224, 224])
torch.Size([32])
torch.Size([32, 10])
torch.Size([32])
2.291752815246582
tensor([7, 9, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 9, 7, 7, 7, 7, 7,
        7, 7, 7, 7, 7, 7, 7, 7], device='cuda:0')
tensor([0, 9, 7, 2, 4, 8, 8, 6, 1, 8, 2, 6, 6, 4, 9, 8, 9, 4, 0, 4, 1, 7, 8, 0,
        4, 3, 7, 6, 7, 0, 9, 8], device='cuda:0')
5


In [ ]:
# 추후 시각화를 위해 매 epoch마다 loss, accuracy 값을 저장할 리스트
seq_train_loss = []
seq_test_loss = []
seq_train_acc = []
seq_test_acc = []

In [ ]:
# train 함수 정의

def train(epoch):
    print(f'\n[ Train epoch: {epoch+1} ]')

    model.train() # train은 항상 이걸 지정하고 시작! - Dropout, Batch Normalization 등의 효과를 적용하고 진행하기 위함

    running_loss = 0.0
    batch_losses = []
    total = 0
    correct = 0

    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device) # dataloader의 input image와 label도 device에 넣어줘야됨
        
        # DL 학습 기본 코드
        optimizer.zero_grad() # gradient 초기화
        outputs = model(inputs) # 현재 batch의 inputs을 모델에 넣어 outputs 추출 (확률값)
        loss = criterion(outputs, targets) # 추출한 outputs와 원래 label인 targets 사이 loss 계산
        loss.backward() # 계산한 loss 기반으로 gradient 값 계산
        optimizer.step() # weight parameter update

        total += targets.size(0) # batch 데이터 개수 더하기
        running_loss += loss.item()
        batch_losses.append(loss.item())
        
        _, predicted = outputs.max(1) # 확률값 가장 높게 나타난 클래스
        correct += (predicted == targets).sum().item() # 현재 batch 내에서 알맞게 분류한 이미지 개수 더하기
        
        if batch_idx % 300 == 299:
            print(f'\nCurrent batch: {str(batch_idx+1)}')
            print(f'Average train loss of recent 300 batches: {running_loss / 300}') # 이렇게 출력하는 것이 꼭 필요한 것은 아니지만, 중간중간 확인을 위해 매우 권장
            running_loss = 0.0

    avg_loss = sum(batch_losses) / len(batch_losses)
    seq_train_loss.append(avg_loss)
    seq_train_acc.append(100*correct/total)
    print('\nTotal train accuarcy:', 100. * correct / total) # 전체 데이터 개수에서 맞게 예측한 비율
    print('Total train loss:', avg_loss)

In [ ]:
# test 함수 정의
def test(epoch):
    print(f'\n[ Test epoch: {epoch+1} ]')

    model.eval() # eval은 항상 이걸 지정하고 시작! - Dropout, Batch Normalization 등의 효과를 적용하지 않기 위함!
                   # ex. evaluation 할때는 Dropout 없이 지금까지 학습한 모든 node를 활용해서 진행해야됨

    loss = 0
    batch_losses = []
    correct = 0
    total = 0

    with torch.no_grad(): # gradient update 안함 - eval과 torch.no_grad는 하나의 세트
        for batch_idx, (inputs, targets) in enumerate(test_loader):
            inputs, targets = inputs.to(device), targets.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, targets)
            batch_losses.append(loss.item())

            total += targets.size(0)
            _, predicted = outputs.max(1)
            correct += (predicted == targets).sum().item()

    avg_loss = sum(batch_losses) / len(batch_losses)
    seq_test_loss.append(avg_loss)
    seq_test_acc.append(100 * correct / total)
    print('\nTest accuarcy:', 100. * correct / total)
    print('Test average loss:', avg_loss)

In [ ]:
# training

num_epoch = 40

for epoch in range(0, num_epoch):
    train(epoch)
    test(epoch)

# model 저장! - parameter값 저장
torch.save(model.state_dict(), './mymodel.pt')
print('Model Saved!')


[ Train epoch: 1 ]

Current batch: 300
Average train loss of recent 300 batches: 2.2158764922618865

Current batch: 600
Average train loss of recent 300 batches: 2.0948877942562105

Total train accuarcy: 22.910450897171913
Total train loss: 2.137061884166092

[ Test epoch: 1 ]

Test accuarcy: 27.100054628734714
Test average loss: 2.0576680318642686

[ Train epoch: 2 ]

Current batch: 300
Average train loss of recent 300 batches: 2.05214643796285

Current batch: 600
Average train loss of recent 300 batches: 2.0004034388065337

Total train accuarcy: 28.516199520948017
Total train loss: 2.0142397588940075

[ Test epoch: 2 ]

Test accuarcy: 31.545993192419214
Test average loss: 1.950546895944944

[ Train epoch: 3 ]

Current batch: 300
Average train loss of recent 300 batches: 1.91959849913915

Current batch: 600
Average train loss of recent 300 batches: 1.8613915689786276

Total train accuarcy: 33.907635416228935
Total train loss: 1.882201093980061

[ Test epoch: 3 ]

Test accuarcy: 35.86

In [ ]:
# loss, accuracy 추이 확인 with plt.plot


import matplotlib.pyplot as plt


In [ ]:
plt.figure(figsize=(10,10))
plt.plot(seq_train_loss, label='Train')
plt.plot(seq_test_loss, label='Test')
plt.legend()
plt.show()


NameError: ignored

<Figure size 720x720 with 0 Axes>

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(seq_train_acc, label='Train')
plt.plot(seq_test_acc, label='Test')
plt.legend()
plt.show()